# load, modify xml file, development

In [4]:
import xml.etree.ElementTree as ET
import os

In [12]:

def update_config(file_in, file_out=None, overwrite=False, **params):
    """ Modify an Ichthyop xml input file
    
    Parameters
    ----------
    file_in: str
        Input file
    file_out: str, optional
        Output file
    overwrite: boolean, optional
    **kwargs:
        parameters to be modified, e.g.
        initial_time="year 2011 month 01 day 01 at 01:00"
        
    """

    if file_out is None:
        file_out = file_in.replace(".xml", "_new.xml")
    
    tree = ET.parse(file_in)
    root = tree.getroot()
    
    modified = {k: False for k in params}

    for p in root.iter("parameter"):
        #print(p)
        #print(dir(p))
        #print(p.items())
        for k in p.iter("key"):
            #if k.text=="initial_time":
            if k.text in params:
                #print(k)
                #print(k.text)
                v = p.find("value")
                #print(v)
                #print(v.text)
                #v.text = "some new date"
                v.text = str(params[k.text])
                modified[k.text] = True

    assert all([b for k, b in modified.items()]), "One or several parameters were not modified"
                
    if overwrite or not os.path.isfile(file_out):
        tree.write(file_out)
        print("File {} has been generated".format(file_out))
    else:
        print("Nothing done")
    

cfg = dict(initial_time="year 2011 month 01 day 01 at 01:00",
           transport_duration="0010 day(s) 00 hour(s) 00 minute(s)",
           number_particles=500,
           lon_stain=-0.4,
           lat_stain=49.5,
           depth_stain=-0.0,
           radius_stain=100,
           thickness_stain=1.0,
          )
    

#update_config("taos_mars3d.xml", initial_time="year 2011 month 01 day 01 at 01:00")
update_config("taos_mars3d.xml", overwrite=True, **cfg)



File taos_mars3d_new.xml has been generated


---

List of relevant parameters:

- initial_time="year 2011 month 01 day 01 at 01:00"
- Transport duration="0010 day(s) 00 hour(s) 00 minute(s)"
- number_particles=50
- lon_stain=-0.4
- lat_stain=49.5
- depth_stain=-0.0
- radius_stain=100 [meters]
- thickness_stain=1.0 [meters]
- 